In [1]:
library(tidyverse)
library(tidymodels)
library(repr)
library(rvest)
library(stringr)
library(dbplyr)
library(DBI)
library(RPostgres)
library(lubridate)
library(RColorBrewer)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

# Algerian Forest Fires and the Weather
## Background and Introduction:
As our climate changes, it is important to understand the conditions which can lead to forest fire ignition. Due to increasing temperatures, summer temperatures have been increasing, resulting in dryer heat (lower humidity), dryer forests, which can lead to forest fires. Rising temperatures result in inequality of temperature between the sky and the earth, leading to further likelihood of lightning--along with fire-related human activites, igniting forces are sure to lead to forest fires. ([Source](https://climateatlas.ca/forest-fires-and-climate-change#:~:text=Dry%20heat%20helps%20create%20more,rising%20temperature%20of%20the%20Arctic.))


## Q: What are the weather conditions needed to result in a forest fire in the Algerian Forest Areas?
## Dataset:
"The Algerian Forest Fires Dataset" ([link](https://archive.ics.uci.edu/ml/datasets/Algerian+Forest+Fires+Dataset++)) provides information on forest fires in two areas: Sidi-Bel Abbas and Bejaia regions. The data was taken from June 2012- September 2012, and contains information on the following data:

1. Date: Day, month, year
*Weather data*

2. Temp: temperature noon (Celsius): 22 to 42
3. RH: Relative Humidity (%): 21 to 90
4. Ws: Wind speed (km/h): 6 to 29
5. Rain: total day (mm): 0 to 16.8

*FWI Components*

6. Fine Fuel Moisture Code (FFMC) index: 28.6 to 92.5
7. Duff Moisture Code (DMC) index: 1.1 to 65.9
8. Drought Code (DC) index: 7 to 220.4
9. Initial Spread Index (ISI): 0 to 18.5
10. Buildup Index (BUI): 1.1 to 68
11. Fire Weather Index (FWI): 0 to 31.1

*Classes*
12. Fire or Not Fire

Fire Weather Index (FWI) is a general measure used to predict the intensity of a potential fire in the area, & takes into account each component when calculating this. 

In [2]:
forest_fires_untidy <- read_csv(file = "https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv", 
                                skip = 1, col_names = TRUE, col_types = cols_only(
                                      day = col_integer(),
                                      month = col_integer(),
                                      year = col_integer(),
                                      Temperature = col_double(),
                                      RH = col_double(),
                                      Ws = col_double(),
                                      Rain = col_double(),
                                      FFMC = col_double(),
                                      DMC = col_double(),
                                      DC = col_double(),
                                      ISI = col_double(),
                                      BUI = col_double(),
                                      FWI = col_double(),
                                      Classes = col_character()))  
  
bejaia_region <- forest_fires_untidy %>%
    slice(1:122) %>%
    mutate(region = "bejaia")

sidi_bel_region <- forest_fires_untidy %>%
    slice(125:246) %>%
    mutate(region = "sidi bel abbes")

forest_fire_data <- rbind(bejaia_region, sidi_bel_region) 

fire_data <- forest_fire_data %>%
     filter(!is.na(Classes)) %>%
     select(day, month, Temperature, FWI, RH, Ws, Rain, Classes) %>%
     mutate(Classes = as_factor(Classes))

fire_data

set.seed(1)
fire_split <- initial_split(fire_data, prop = 0.75, strata = Classes)
fire_train <- training(fire_split)
fire_test <- testing(fire_split)

fire_recipe <- recipe(Classes ~ month + Temperature + FWI, data = fire_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

set.seed(1)
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) %>%
    set_engine("kknn") %>%
    set_mode("classification")

knn_fit <- workflow() %>%
    add_recipe(fire_recipe) %>%
    add_model(knn_spec) %>%
    fit(data = fire_train)

fire_test_predictions <- predict(knn_fit, fire_test) %>%
    bind_cols(fire_test)

fire_train %>%
    group_by(Classes) %>%
    summarize(n = n(),
    average_temp = mean(Temperature),
    average_FWI = mean(FWI))

Warning message:
“18 parsing failures.
row   col   expected                        actual                                                                                                       file
123 day   an integer Sidi-Bel Abbes Region Dataset 'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
123 NA    14 columns 1 columns                     'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 day   an integer day                           'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 month an integer month                         'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 year  an integer year                          'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
... ..... ...

day,month,Temperature,FWI,RH,Ws,Rain,Classes
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,29,0.5,57,18,0.0,not fire
2,6,29,0.4,61,13,1.3,not fire
3,6,26,0.1,82,22,13.1,not fire
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
28,9,27,0.2,87,29,0.5,not fire
29,9,24,0.7,54,18,0.1,not fire
30,9,24,0.5,64,15,0.2,not fire


`summarise()` ungrouping output (override with `.groups` argument)



Classes,n,average_temp,average_FWI
<fct>,<int>,<dbl>,<dbl>
not fire,80,29.97500,0.92375
fire,103,33.70874,11.84854
